In [1]:
import pickle
import pandas as pd
import numpy as np
import os
d = "raw/pk/"
files = sorted([d + f for f in os.listdir(d) if f.endswith("lpow")])
print(files)

['raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9001_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9001_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9002_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9002_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9003_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9003_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9004_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9004_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9005_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9005_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9006_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9006_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9007_flip_v3.0_n200rand.lpow', 'raw/mock_gsafe0DR7VACNGCm_abM21.2_gr8.0_R9007_v3.0_n200rand.lpow', 'raw/mock_gs

In [2]:
def getdf(loc):
    df = pd.read_csv(loc, comment="#", delim_whitespace=True, names=["s", "xi", "xi0", "xi2", "xi4"])
    return masked.astype(np.float32)

res = {f.lower(): getdf(f) for f in files}
split = {
    "pre-recon": [v for k, v in res.items() if "recon" not in k],
    "post-recon": [v for k, v in res.items() if "recon" in k]
}

In [5]:
with open("mock_lpow.pkl", "wb") as f:
    pickle.dump(split, f)

In [3]:
a = split["pre-recon"][0]

In [4]:
a.shape

(249, 3)